In [ ]:
!pip install rake-nltk

In [2]:
import numpy as np
import pandas as pd
from rake_nltk import Rake
import sklearn
import nltk

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

In [4]:
dfanime = pd.read_json('/content/drive/MyDrive/anime_data.json')

In [ ]:
dfanime.head()

In [6]:
dfanime['genres'] = dfanime['genres'].apply(lambda x: ', '.join([f"'{genre['name']}'" for genre in x]) if isinstance(x, list) else '')

In [ ]:
dfanime.head()

In [ ]:
syp = dfanime['synopsis']
genres = dfanime['genres']
title = dfanime['title']
for i, t in enumerate(title):
    title[i] = t.lower()

In [17]:
title.head()

0                       cowboy bebop
1    cowboy bebop: tengoku no tobira
2                             trigun
3                 witch hunter robin
4                     bouken ou beet
Name: title, dtype: object

In [ ]:
syp = syp.fillna(' ')
for i,each_anime_genre in enumerate(genres):
    each_anime_genre = each_anime_genre.replace("'","")
    each_anime_genre = each_anime_genre.replace(",","")
    genres.iloc[i] = each_anime_genre

In [ ]:
genres

In [ ]:
rake = Rake()
keywords = []
for plot in syp:
    rake.extract_keywords_from_text(plot)
    keywords_i = rake.get_ranked_phrases()
    keywords_i_string = ""
    for keyword in keywords_i:
        keywords_i_string = keywords_i_string + " " + keyword
    keywords.append(keywords_i_string)
dfanime['keywords'] = keywords
dfanime['keywords'][0]

In [13]:
dfanime['bag of words'] = dfanime['genres'] + dfanime['keywords']

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
vectorizer = CountVectorizer()
vectorized_bag_of_words = vectorizer.fit_transform(dfanime['bag of words'])
vectorized_bag_of_words = vectorized_bag_of_words.toarray()

In [15]:
def recommend(show_title, n_recom):
    similarity_matrix = cosine_similarity(vectorized_bag_of_words, vectorized_bag_of_words[list(np.where(dfanime["title"] == show_title)[0]), :])
    similarity_dataframe = pd.DataFrame(similarity_matrix)
    similarity_dataframe.index = title 
    similarity_dataframe =  similarity_dataframe.iloc[:,0]
    similarity_dataframe = similarity_dataframe.sort_values(ascending = False)
    similarity_dataframe = similarity_dataframe.drop_duplicates()
    return list(similarity_dataframe.index)[1:n_recom + 1]

In [21]:
recommend("cowboy bebop", 8)

['cowboy bebop: tengoku no tobira',
 'saraba uchuu senkan yamato: ai no senshi-tachi',
 'planetes picture drama',
 'space cobra pilot',
 'cobra the animation: time drive',
 'kidou shinseiki gundam x',
 'uchuu senkan yamato',
 'tsumiki no ie']